In [8]:
import numpy as np
import os
import sys
import torch
from torch.profiler import profile, ProfilerActivity

sys.path.append(os.path.join(os.getenv("HOME"), "RNN_Manifold/"))
import encoder_decoder_core, decode_1d


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using {}".format(device))

using cuda


The purpose of this notebook is to get an idea of the resource use of the main decoder functions to make sure nothing weird is going on.

In [5]:
batch_size = 1000
embedded_dim = 20
manifold_dim = 1
decoder_hidden_size = 1000
encoder_hidden_size = int(1.5 * decoder_hidden_size)
decoder_n_hidden = 2
encoder_n_hidden = decoder_n_hidden

fake_data = np.random.uniform(-1, 1, (batch_size, embedded_dim))
fake_data_torch = torch.tensor(fake_data, dtype=torch.get_default_dtype()).to(device)

In [6]:
encoder_net = encoder_decoder_core.Encoder1D(embedded_dim, manifold_dim, 0, encoder_hidden_size, encoder_n_hidden).to(device)
decoder_net = encoder_decoder_core.Decoder1D(embedded_dim, manifold_dim, 0, decoder_hidden_size, decoder_n_hidden).to(device)

/home/trevor/brainvenv/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [10]:
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    decoded_angles, re_encoded_points, _ = decode_1d.decode_encode_cost(decoder_net, encoder_net, fake_data_torch)

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=20))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             aten::tanh         8.23%     121.000us        31.88%     469.000us      58.625us      67.000us         7.15%      67.000us       8.375us           0 b           0 b      31.86 Mb      31.86 M

STAGE:2023-01-19 14:16:26 305722:305722 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-19 14:16:26 305722:305722 ActivityProfilerController.cpp:300] Completed Stage: Collection


In [11]:
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True, profile_memory=True) as prof:
    _ = decode_1d.distance_costs(encoder_net, re_encoded_points, decoded_angles, 20)

print(prof.key_averages().table(sort_by="self_cuda_memory_usage", row_limit=20))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             aten::tanh         0.49%      73.000us         3.84%     566.000us     141.500us     887.000us         7.55%     887.000us     221.750us           0 b           0 b     344.10 Mb     344.10 M

STAGE:2023-01-19 14:16:33 305722:305722 ActivityProfilerController.cpp:294] Completed Stage: Warm Up
STAGE:2023-01-19 14:16:33 305722:305722 ActivityProfilerController.cpp:300] Completed Stage: Collection
